In [23]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from tqdm import tqdm
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [24]:
# Create 10000 samples for TTVFast
size = 10000
mu1 = 1e-5; mu2 = 1e-5
e1 = 10**np.random.uniform(low=-3, high=-1, size=size); e2 = e1.copy()
w1 = np.random.uniform(low=0, high=360, size=size)
w2 = (w1.copy() + np.random.choice([0, 180], size=size)) % 360
Delta = np.random.uniform(low=-0.01, high=0.01, size=size)
j = 5; N = 2
P1 = 30; P2 = P1 * (Delta + 1) * j / (j-N)

# Compile to dataframe
df = pd.DataFrame({
    'e1': e1, 'w1': w1,
    'e2': e2, 'w2': w2,
    'Delta': Delta, 'j': j, 'N': N,
    'P1': P1, 'P2': P2,
    'mu1': mu1, 'mu2': mu2,
    'valid': True
})
df.head()

e1          w1        e2          w2     Delta  j  N  P1         P2  \
0  0.007380  324.612682  0.007380  324.612682 -0.004655  5  2  30  49.767240   
1  0.030366   87.858502  0.030366  267.858502 -0.009659  5  2  30  49.517031   
2  0.001313   27.412969  0.001313   27.412969 -0.003028  5  2  30  49.848618   
3  0.075404   82.673787  0.075404   82.673787  0.001778  5  2  30  50.088879   
4  0.015761  261.026633  0.015761   81.026633  0.006085  5  2  30  50.304259   

       mu1      mu2  valid  
0  0.00001  0.00001   True  
1  0.00001  0.00001   True  
2  0.00001  0.00001   True  
3  0.00001  0.00001   True  
4  0.00001  0.00001   True

In [25]:
# Copy dataframe for fitting results
df_fit = df.copy()
df_fit_cols = ["A1", "B1", "V1", "Pttv1", "phase1", "std1", "A2", "B2", "V2", "Pttv2", "phase2", "std2", "R2_1", "R2_2"]
df_fit[df_fit_cols] = np.nan
df_fit.head()

e1          w1        e2          w2     Delta  j  N  P1         P2  \
0  0.007380  324.612682  0.007380  324.612682 -0.004655  5  2  30  49.767240   
1  0.030366   87.858502  0.030366  267.858502 -0.009659  5  2  30  49.517031   
2  0.001313   27.412969  0.001313   27.412969 -0.003028  5  2  30  49.848618   
3  0.075404   82.673787  0.075404   82.673787  0.001778  5  2  30  50.088879   
4  0.015761  261.026633  0.015761   81.026633  0.006085  5  2  30  50.304259   

       mu1  ...  phase1  std1  A2  B2  V2  Pttv2  phase2  std2  R2_1  R2_2  
0  0.00001  ...     NaN   NaN NaN NaN NaN    NaN     NaN   NaN   NaN   NaN  
1  0.00001  ...     NaN   NaN NaN NaN NaN    NaN     NaN   NaN   NaN   NaN  
2  0.00001  ...     NaN   NaN NaN NaN NaN    NaN     NaN   NaN   NaN   NaN  
3  0.00001  ...     NaN   NaN NaN NaN NaN    NaN     NaN   NaN   NaN   NaN  
4  0.00001  ...     NaN   NaN NaN NaN NaN    NaN     NaN   NaN   NaN   NaN  

[5 rows x 26 columns]

In [26]:
# TTVFast simulation (Copypasted from 1st order code)
# Create TTV simulations and extract Amplitude and Period
gravity = 0.000295994511 # AU^3/day^2/M_sun
stellar_mass = 1.0       # M_sun

for i, row in tqdm(df.iterrows(), total=len(df)):  
    # Initialize variables
    P1 = row['P1']; P2 = row['P2']
    j = row['j']; N = row['N']
    PTTV = 1/np.abs(j/P2 - (j-N)/P1)
    
    e1 = row['e1']; w1 = row['w1']
    e2 = row['e2']; w2 = row['w2']
    mu1 = row['mu1']; mu2 = row['mu2']
    
    # Create TTVFast planets
    planet1 = models.Planet(
        mass=mu1,            # M_sun
        period=P1,           # days
        eccentricity=e1,
        inclination=90,      # degrees
        longnode=0,          # degrees
        argument=90+w1,      # degrees
        mean_anomaly=0       # degrees
    )
    planet2 = models.Planet(
        mass=mu2,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=0,
        argument=90+w2,
        mean_anomaly=0,
    )
    planets = [planet1, planet2]
    
    begin_time = -P1 * 50
    Time = begin_time   # days
    dt = P1 / 100       # days
    Total = 1600        # days
    
    results = ttvfast.ttvfast(planets, stellar_mass, Time, dt, Total,
                              input_flag=1)

    # Create TTV signal from TTVFast Output
    out = pd.DataFrame(results['positions']).transpose()
    out.columns = ['planet', 'epoch', 'times', 'rsky', 'vsky']
    
    table0 = out[(out.planet == 0) & (out.times > 0)]
    n0 = table0.epoch
    tn0 = table0.times

    table1 = out[(out.planet == 1) & (out.times > 0)]
    n1 = table1.epoch
    tn1 = table1.times

    # If few TTVFast output, skip
    len_threshold = 10
    if (len(n0) <= len_threshold) or (len(n1) <= len_threshold):
        continue
        
    time0, ttv0, ttv0_err = ttvfit.return_ttv(epochs=n0, midtransits=tn0)
    time1, ttv1, ttv1_err = ttvfit.return_ttv(epochs=n1, midtransits=tn1)
    
    # Make sure ttv0 and ttv1 actually are centered around O-C = 0
    ttv0 = ttv0 - (np.max(ttv0) + np.min(ttv0)) / 2
    ttv1 = ttv1 - (np.max(ttv1) + np.min(ttv1)) / 2
    
    # Get sinusoidal TTV model parameters
    popt_ttv0, perr_ttv0, r2_0 = ttvfit.characterize_ttv(time=time0, ttv=ttv0, ttv_err=ttv0_err,
                                                         expected_period=PTTV, method='lmfit')
    popt_ttv1, perr_ttv1, r2_1 = ttvfit.characterize_ttv(time=time1, ttv=ttv1, ttv_err=ttv1_err,
                                                         expected_period=PTTV, method='lmfit')
    std0, std1 = np.std(ttv0), np.std(ttv1)

    # Write results to df_fit
    df_fit.loc[i, df_fit_cols]  = [*popt_ttv0, std0, *popt_ttv1, std1, r2_0, r2_1]

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [25:27<00:00,  6.55it/s]


In [27]:
# If output fit has no values, convert valid from TRUE to FALSE
output_has_nan = df_fit[df_fit_cols].isna().any(axis=1)
df_fit.loc[output_has_nan, 'valid'] = False

# If output has PTTV wildly different from each other, convert valid from TRUE to FALSE
Pttv_ratio_threshold = 1.05
Pttv_not_similar = (df_fit['Pttv1'] / df_fit['Pttv2'] > Pttv_ratio_threshold) | (df_fit['Pttv2'] / df_fit['Pttv1'] > Pttv_ratio_threshold)
df_fit.loc[Pttv_not_similar, 'valid'] = False

df_fit.head()

e1          w1        e2          w2     Delta  j  N  P1         P2  \
0  0.007380  324.612682  0.007380  324.612682 -0.004655  5  2  30  49.767240   
1  0.030366   87.858502  0.030366  267.858502 -0.009659  5  2  30  49.517031   
2  0.001313   27.412969  0.001313   27.412969 -0.003028  5  2  30  49.848618   
3  0.075404   82.673787  0.075404   82.673787  0.001778  5  2  30  50.088879   
4  0.015761  261.026633  0.015761   81.026633  0.006085  5  2  30  50.304259   

       mu1  ...    phase1      std1        A2            B2        V2  \
0  0.00001  ...  6.017048  0.000746  0.032766 -4.888214e-05  0.171238   
1  0.00001  ...  2.536053  0.004303  0.001056 -8.205480e-07  0.008961   
2  0.00001  ...  6.393856  0.000787 -0.006181 -7.546527e-06  0.036926   
3  0.00001  ...  2.921880  0.001042  0.654530 -6.932827e-04  2.771991   
4  0.00001  ...  2.662700  0.002068 -0.006806  9.329151e-06  0.006680   

          Pttv2    phase2      std2      R2_1      R2_2  
0  21876.230874  6.090359  0.000712  0.006857  0.011394  
1   1047.864761  5.628309  0.006179  0.959770  0.969121  
2  28883.964329  6.447432  0.000764  0.004168  0.005261  
3  25008.344610  6.045786  0.001114  0.368095  0.460295  
4   1757.656951  6.238018  0.002687  0.871939  0.933145  

[5 rows x 26 columns]

In [28]:
# Export Outputs
df_fit.to_csv("deckagol_TTVFast_fits.csv", index=False)